In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle 

In [2]:
# Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocess the data
# Drop irrelevant columns
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

In [4]:
# Encoding
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [5]:
# OneHotEncode 'Geography' column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoded_value = onehot_encoder_geo.fit_transform(data[['Geography']])

In [6]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoded_value.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [8]:
# Combine all the columns with the original data
data = pd.concat([data.drop(columns='Geography'), geo_encoded_df], axis=1)

In [9]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
# Save the encoders and scaler so that we can use it in the Application while creating
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [11]:
# Divide the dataset into dependent and independent features
X = data.drop(columns='Exited')
y = data['Exited']

In [12]:
# Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# ANN Implementation

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [16]:
# Build our ANN model
## Creating the hidden layers and the output
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  ## 1st Hidden Layer and it is connected with input layer
    Dense(32, activation='relu'),   ## HL2
    Dense(1, activation='sigmoid')  ## OutPut layer
]
)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [41]:
## Compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [42]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)   # Tensorboard will visualize all the logs while training the model

In [43]:
## Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [44]:
## Training the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 736us/step - loss: 30.7361 - accuracy: 0.7276 - val_loss: 9.9276 - val_accuracy: 0.8085
Epoch 2/100
250/250 [==============================] - 0s 512us/step - loss: 29.8442 - accuracy: 0.7337 - val_loss: 20.3560 - val_accuracy: 0.8145
Epoch 3/100
250/250 [==============================] - 0s 504us/step - loss: 22.9501 - accuracy: 0.7408 - val_loss: 18.1446 - val_accuracy: 0.7740
Epoch 4/100
250/250 [==============================] - 0s 566us/step - loss: 32.4034 - accuracy: 0.7366 - val_loss: 39.6457 - val_accuracy: 0.8210
Epoch 5/100
250/250 [==============================] - 0s 509us/step - loss: 39.1090 - accuracy: 0.7391 - val_loss: 30.8344 - val_accuracy: 0.8185
Epoch 6/100
250/250 [==============================] - 0s 510us/step - loss: 29.2540 - accuracy: 0.7471 - val_loss: 19.5429 - val_accuracy: 0.8180
Epoch 7/100
250/250 [==============================] - 0s 506us/step - loss: 25.7297 - accuracy: 0.7436 - val_loss: 57.

In [45]:
model.save('model.h5')

/Users/rowshansiddiki/ANN/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [47]:
# Load TensorNoard Extension
%load_ext tensorboard

In [55]:
%tensorboard --logdir logs/fit20250521-094011

Reusing TensorBoard on port 6008 (pid 1956), started 0:00:09 ago. (Use '!kill 1956' to kill it.)

In [56]:
data.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')